In [12]:
from pymongo.operations import SearchIndexModel

# Create your index model, then create the search index
search_index_model = SearchIndexModel(
  definition = {
    "fields": [
      {
        "type": "vector",
        "path": "embedding",
        "similarity": "cosine",
        "numDimensions": 1024
      }
    ]
  },
  name="vector_index",
  type="vectorSearch",
)
collection.create_search_index(model=search_index_model)

'vector_index'

In [11]:
collection.drop_search_index("vector_index")

OperationFailure: Index vector_index not found in medical_db.medical_knowledge, full error: {'ok': 0.0, 'errmsg': 'Index vector_index not found in medical_db.medical_knowledge', 'code': 27, 'codeName': 'IndexNotFound', '$clusterTime': {'clusterTime': Timestamp(1733748162, 1), 'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'keyId': 0}}, 'operationTime': Timestamp(1733748162, 1)}

In [2]:
import os

from pymongo.mongo_client import MongoClient


uri = f"mongodb://localhost:35533/?directConnection=true&serverSelectionTimeoutMS=2000"

# Create a new client and connect to the server
client = MongoClient(uri)
client.admin.command("ping")

collection = client['medical_db']['medical_knowledge']

In [3]:
from transformers import BertForMaskedLM, BertTokenizer
import torch

tokenizer=BertTokenizer.from_pretrained("IDEA-CCNL/Erlangshen-TCBert-330M-Sentence-Embedding-Chinese")
model=BertForMaskedLM.from_pretrained("IDEA-CCNL/Erlangshen-TCBert-330M-Sentence-Embedding-Chinese")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at IDEA-CCNL/Erlangshen-TCBert-330M-Sentence-Embedding-Chinese were not used when initializing BertForMaskedLM: ['bert.pooler.d

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 1024, padding_idx=1)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-

In [4]:
import json

with open("medical_knowledge_m.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [5]:
def get_embedding(data):
    with torch.no_grad():
        data_input = tokenizer(data, return_tensors="pt", padding=True).to(device)  # 数据也要移动到GPU
        data_output = model(**data_input, output_hidden_states=True)
        embedding = torch.mean(data_output.hidden_states[-1], dim=1).squeeze()
        return embedding.tolist()

In [8]:
batch_size = 2000
for i in range(0, len(data), batch_size):
    batch = data[i:i+batch_size]
    instructions = [item["instruction"] for item in batch]
    embeddings =  get_embedding(instructions)
    for item, embedding in zip(batch, embeddings):
        item["embedding"] = embedding

    collection.insert_many(batch)  # 批量插入
    print(f"已插入第 {i // batch_size + 1} 批数据")
    

已插入第 1 批数据
已插入第 2 批数据
已插入第 3 批数据
已插入第 4 批数据
已插入第 5 批数据
已插入第 6 批数据
已插入第 7 批数据
已插入第 8 批数据
已插入第 9 批数据
已插入第 10 批数据
已插入第 11 批数据
已插入第 12 批数据
已插入第 13 批数据
已插入第 14 批数据
已插入第 15 批数据
已插入第 16 批数据
已插入第 17 批数据
已插入第 18 批数据
已插入第 19 批数据
已插入第 20 批数据
已插入第 21 批数据
已插入第 22 批数据
已插入第 23 批数据
已插入第 24 批数据
已插入第 25 批数据
已插入第 26 批数据
已插入第 27 批数据
已插入第 28 批数据
已插入第 29 批数据
已插入第 30 批数据
已插入第 31 批数据
已插入第 32 批数据
已插入第 33 批数据
已插入第 34 批数据
已插入第 35 批数据
已插入第 36 批数据
已插入第 37 批数据
已插入第 38 批数据
已插入第 39 批数据
已插入第 40 批数据
已插入第 41 批数据
已插入第 42 批数据
已插入第 43 批数据
已插入第 44 批数据
已插入第 45 批数据
已插入第 46 批数据
已插入第 47 批数据
已插入第 48 批数据
已插入第 49 批数据
已插入第 50 批数据
已插入第 51 批数据
已插入第 52 批数据
已插入第 53 批数据
已插入第 54 批数据
已插入第 55 批数据
已插入第 56 批数据
已插入第 57 批数据
已插入第 58 批数据
已插入第 59 批数据
已插入第 60 批数据
已插入第 61 批数据
已插入第 62 批数据
已插入第 63 批数据
已插入第 64 批数据
已插入第 65 批数据
已插入第 66 批数据
已插入第 67 批数据
已插入第 68 批数据
已插入第 69 批数据
已插入第 70 批数据
已插入第 71 批数据
已插入第 72 批数据
已插入第 73 批数据
已插入第 74 批数据
已插入第 75 批数据
已插入第 76 批数据
已插入第 77 批数据
已插入第 78 批数据
已插入第 79 批数据
已插入第 80 批数据
已插入第 81 批数据
已插入第 82 批数据
已插入第 83 批数据
已插入第 84 批数据
已

In [15]:
def get_query_results(query):
    query_embedding = get_embedding(query)

    pipeline = [
       {
          "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "exact": True,
                "limit": 5
          }
       }, 
       {
          "$project": {
             "_id": 0, 
             "instruction": 1,
              "input": 1,
              "output": 1,
             "score": {
                "$meta": "vectorSearchScore"
             }
          }
       },
        {
            "$addFields": {
                "result": {
                    "$concat": [
                        "主题：",
                        "$instruction",
                        "\n患者描述：",
                        "$input",
                        "\n医生诊断：",
                        "$output",
                        "\n"
                    ]
                }
            }
        }
    ]
    
    return collection.aggregate(pipeline)

query = "肥胖"
results = get_query_results(query)
for res in results:
    print(f"{res['result']}, score:{res['score']}")

主题：孩子肥胖懒应怎样医治
患者描述：男孩子，刚2岁，今年，察觉到，孩子身上越来越肉乎了，而且，吃的饭也比一般孩子多，平时都不喜欢吃去玩，请问：孩子肥胖懒应怎样医治。
医生诊断：小儿肥胖你可以先给孩子制定运动量和改变饮食，像游泳，爬坡这类游泳运动对肥胖的症状都很好的效果，并且让孩子多吃一些菠菜，胡萝卜等这些高纤维的食物，禁止孩子吃高热量，高脂肪的食物，像蛋糕，干果，曲奇饼干等，严格的控制孩子的饮食，不要让他暴饮暴食，多运动对改变孩子肥胖都是有好处的，在给孩子治疗肥胖的期间，如果孩子情况严重一定要带孩子去正规的医院治疗。
, score:0.8015985488891602
主题：孩子肥胖爱吃肉理应怎样治疗
患者描述：男宝，已经8岁，近期，观察到，我家孩子越来越胖了，同时，最近越来越能吃了，还特别的懒，请问：孩子肥胖爱吃肉理应怎样治疗。
医生诊断：如果孩子得了肥胖症的话家长要通过运动跟饮食改变他现在的情况，给孩子在承受范围内安排孩子游泳，慢跑等运动，并且多让孩子吃一些高纤维食物食物，像菠菜，胡萝卜等，瓜子，汉堡这样高热量的食物就不要让孩子吃了，也不要让孩子总是暴饮暴食，让他多运动，肥胖症治疗期间家长要根据孩子的情况进行合理的治疗，如果病情严重的话一定要去医院查明原因针对治疗。
, score:0.8009512424468994
主题：孩子肥胖懒应怎样治疗
患者描述：我家男宝，今年9岁，最近几周，觉得，我家孩子身上肉很多，另外，最近越来越能吃了，还特别的懒，请问：孩子肥胖懒应怎样治疗。
医生诊断：孩子身体肥胖可以先让孩子增加与运动量和控制饮食来缓解症状，可以给孩子安排游泳或者慢跑这样的有氧运动，并且多让孩子吃一些高纤维食物食物，像菠菜，胡萝卜等，禁止孩子吃一些油炸食品和干果类食物，这些都是干热量高脂肪的食物，生活中不要让孩子在床上吃零食或者吃完就躺着这些不好的习惯也会让脂肪堆积，在治疗小儿肥胖在期间，家长如果发现孩子情况严重要及时去医院查明原因在针对性的治疗。
, score:0.800765872001648
主题：孩子肥胖懒应怎样治疗好
患者描述：男孩子，刚3岁，最近几周，发现，我家孩子越来越胖了，同时，食量非常的大，还特别的懒，请问：孩子肥胖懒应怎样治疗好。
医生诊断：孩子出现肥胖症的情况。家长要通过孩子运功和健康的饮食来缓解他的症状，给孩子在承受范围内安排孩子

In [6]:
collection.delete_many({})

DeleteResult({'n': 16000, 'electionId': ObjectId('7fffffff0000000000000002'), 'opTime': {'ts': Timestamp(1733748693, 225), 't': 2}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1733748693, 225), 'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'keyId': 0}}, 'operationTime': Timestamp(1733748693, 225)}, acknowledged=True)

In [7]:
results = collection.find({})
print(len(list(results)))

0


In [6]:
import json

with open("medical_knowledge.json", "r", encoding="utf-8") as file:
    data = json.load(file)

batch_size = 100
for i in range(0, len(data), batch_size):
    batch = data[i:i+batch_size]  # 获取当前批次数据
    collection.insert_many(batch)  # 批量插入
    print(f"已插入第 {i // batch_size + 1} 批数据")

已插入第 1 批数据
已插入第 2 批数据
已插入第 3 批数据
已插入第 4 批数据
已插入第 5 批数据
已插入第 6 批数据
已插入第 7 批数据
已插入第 8 批数据
已插入第 9 批数据
已插入第 10 批数据
已插入第 11 批数据
已插入第 12 批数据
已插入第 13 批数据
已插入第 14 批数据
已插入第 15 批数据
已插入第 16 批数据
已插入第 17 批数据
已插入第 18 批数据
已插入第 19 批数据
已插入第 20 批数据
已插入第 21 批数据
已插入第 22 批数据
已插入第 23 批数据
已插入第 24 批数据
已插入第 25 批数据
已插入第 26 批数据
已插入第 27 批数据
已插入第 28 批数据
已插入第 29 批数据
已插入第 30 批数据
已插入第 31 批数据
已插入第 32 批数据
已插入第 33 批数据
已插入第 34 批数据
已插入第 35 批数据
已插入第 36 批数据
已插入第 37 批数据
已插入第 38 批数据
已插入第 39 批数据
已插入第 40 批数据
已插入第 41 批数据
已插入第 42 批数据
已插入第 43 批数据
已插入第 44 批数据
已插入第 45 批数据
已插入第 46 批数据
已插入第 47 批数据
已插入第 48 批数据
已插入第 49 批数据
已插入第 50 批数据
已插入第 51 批数据
已插入第 52 批数据
已插入第 53 批数据
已插入第 54 批数据
已插入第 55 批数据
已插入第 56 批数据
已插入第 57 批数据
已插入第 58 批数据
已插入第 59 批数据
已插入第 60 批数据
已插入第 61 批数据
已插入第 62 批数据
已插入第 63 批数据
已插入第 64 批数据
已插入第 65 批数据
已插入第 66 批数据
已插入第 67 批数据
已插入第 68 批数据
已插入第 69 批数据
已插入第 70 批数据
已插入第 71 批数据
已插入第 72 批数据
已插入第 73 批数据
已插入第 74 批数据
已插入第 75 批数据
已插入第 76 批数据
已插入第 77 批数据
已插入第 78 批数据
已插入第 79 批数据
已插入第 80 批数据
已插入第 81 批数据
已插入第 82 批数据
已插入第 83 批数据
已插入第 84 批数据
已

KeyboardInterrupt: 

In [23]:
collection.delete_many({})

DeleteResult({'n': 25777, 'electionId': ObjectId('7fffffff000000000000007c'), 'opTime': {'ts': Timestamp(1733671685, 935), 't': 124}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1733671685, 935), 'signature': {'hash': b'\xbc\xd78\xbfa\xa7\x18\x97~\x7f[\x89\xefc\xa9\x0br)SZ', 'keyId': 7413746205873668113}}, 'operationTime': Timestamp(1733671685, 935)}, acknowledged=True)

In [1]:
import os
print(os.environ.get('LD_LIBRARY_PATH'))

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


device(type='cuda')